In [ ]:
from test import test
from options.options import Options
from copy import deepcopy
import data

org_opt = Options('options/test_options.yaml')
org_opt.initialize()
org_opt.train_mode = 'GAN'
org_opt.how_many = 8
org_opt.batch_size = org_opt.how_many
dataloader, dataset = data.create_dataloader(org_opt, 'train')
mesh_files = dataset.mesh_paths[:org_opt.how_many]

for exp in range(org_opt.number_of_experiments-1,-1,-1):
    opt = deepcopy(org_opt)
    exp_str = '(exp'+str(exp+1)+')'
    if opt.number_of_experiments<=1:
        exp_str = ''
    opt.set_experiment(exp)
        
    for inst in range(org_opt.number_of_instances):
        inst_str = '(inst'+str(inst+1)+')'
        if opt.number_of_instances==1:
            inst_str = ''
        opt.name = org_opt.name+exp_str+inst_str

        to_print = '***************************************************************************************' + \
                   '\n    Testing results of ' + opt.name + '\n' + \
                   '***************************************************************************************'
        print(to_print)
        exp = test(opt, True)
        exp.run_test(mesh_files, [3,6])

In [ ]:
exp.slice_metrics

In [ ]:
import data
from test_all import test
from options.options import Options
from copy import deepcopy

In [ ]:
org_opt = Options('options/test_options.yaml')
org_opt.initialize()
org_opt.train_mode = 'GAN'
org_opt.batch_size = org_opt.how_many
dataloader, dataset = data.create_dataloader(org_opt)

mesh_files = dataset.mesh_paths[:org_opt.how_many]
data = next(iter(dataloader))
patches = data['mesh_slices'][:org_opt.how_many,0]

opt = deepcopy(org_opt)

exp = 8
inst = 1

# load the dataset
opt = deepcopy(org_opt)
opt.name += '(exp'+str(exp)+')'#(inst'+str(inst)+')'

In [ ]:
t = test(opt, True)
t.run_test(mesh_files, [10,20])

In [ ]:
import torch

In [ ]:
x = [1,2,3,4]
import numpy as np

In [ ]:
np.mean(1)

In [ ]:
import numpy as np
import itertools
from functools import partial
from copy import deepcopy

from util.util import mkdir
from pathlib import Path
import imageio
import torchvision.utils as vutils

from util.mesh_handler import *
from util.util import tensorize_dict

import torch
import torch.nn.functional as F

from scipy import stats

import data
from options.options import Options
from models.pix2pix_model import Pix2PixModel

from util.mesh_handler import *

In [ ]:
x = [1,2,3]+[4,5,6]

In [ ]:
def sample_patches(vox, k, d, padding=True):
    x = torch.Tensor(vox)
    kc, kh, kw = k  # kernel size
    dc, dh, dw = d  # stride

    # Pad to multiples of 32
    if padding:
        x = F.pad(x, (kw//2, kw//2,
                      kh//2, kh//2,
                      kc//2, kc//2))

    patches = x.unfold(1, kc, dc).unfold(2, kh, dh).unfold(3, kw, dw)
    patches = patches.contiguous().view(len(vox),-1,1,kc,kh,kw)
    return patches

def no_padding(input, sz, dim='3D'):
    beg = np.asarray(sz)//2
    end = -1*beg.copy()
    end[end==0] = 1
    
    return input[:,:,beg[0]:end[0], beg[1]:end[1], beg[2]:end[2]]

def generate(vox, sigmas, background, p_blobs, powers, frames):
    out = np.zeros([len(powers),1, *vox.shape])
    blur = blur_vox(vox,sigmas,background)
    
    for i in range(len(powers)): 
        fake = add_blobs(np.copy(blur), p_blobs, background)
        fake = add_poisson_noise(fake, powers[i], frames[i])
        out[i] = fake
        
    return torch.Tensor(out), blur>background

def test(model, vox, powers, frames):
    n_samples = vox.shape[1]
    fake = torch.zeros([len(powers), *vox.shape[1:]])
    fake_mu = torch.zeros_like(fake)
    fake_sigma = torch.zeros_like(fake)
    
    for i in range(len(powers)):
        data = {'mesh_semantics': vox[0],
                'power': n_samples*[powers[i]],
                'frames': n_samples*[frames[i]]}
        data = tensorize_dict(data)
        
        fake[i], fake_mu[i], fake_sigma[i] = model(data,'inference')
    return fake, fake_mu, fake_sigma

def histogram_intersection(fake, real, bins=20):
    sm_ls = []
    
    for i in range(len(real)):
        d1 = fake[i]
        d2 = real[i]
        
        h1 = np.histogram(d1, bins=20, range=(0,1))[0]/len(d1)
        h2 = np.histogram(d2, bins=20, range=(0,1))[0]/len(d2)

        sm = 0
        for i in range(bins):
            sm += min(h1[i], h2[i])
        sm_ls += [sm]
    return sm_ls

In [ ]:
org_opt = Options('options/test_options.yaml')
org_opt.initialize()
org_opt.train_mode = 'GAN'
org_opt.batch_size = org_opt.how_many
dataloader, dataset = data.create_dataloader(org_opt)

mesh_files = dataset.mesh_paths[:org_opt.how_many]
data = next(iter(dataloader))
patches = data['mesh_slices'][:org_opt.how_many,0]

opt = deepcopy(org_opt)

exp = 5
inst = 1

# load the dataset
opt = deepcopy(org_opt)
opt.name += '(exp'+str(exp)+')'#(inst'+str(inst)+')'

In [ ]:
opt = opt
paired = True

out_dir = opt.results_dir+opt.name+'/'
z_pad = opt.delta_slice*(opt.in_Gslices//2)

mkdir(out_dir)

model = Pix2PixModel(opt)
model.eval()

In [ ]:
style_combs = np.asarray(list(itertools.product(opt.powers, opt.frames)))

mesh_file = mesh_files[0]
out_file = out_dir+Path(mesh_file).stem

mesh = Mesh(mesh_file)
mesh.scale_z(min(opt.mesh_res[1:])/opt.mesh_res[0])

vox = mesh.voxelize(opt.mesh_res, z_pad, opt.max_xy_sz, opt.in_dim)
if paired:
    gt, dendrites = generate(vox, opt.sigmas, opt.background, opt.p_blobs, style_combs[:,0], style_combs[:,1])
    
z_ind = np.argmax(vox.sum(-1).sum(-1))

vox_slice = vox[None, None, z_ind-z_pad:z_ind+z_pad+1]
gt_slice = gt[:,:,z_ind-z_pad:z_ind+z_pad+1]
dends_slice = dendrites[None, None, z_ind-z_pad:z_ind+z_pad+1]
background_slice = ~dends_slice

block = [1, *opt.crop_xy_sz]
vox_patches = sample_patches(vox_slice[0], block, block)
gt_patches = sample_patches(gt_slice[:,0], block, block)
dend_patches = sample_patches(dends_slice[0], block, block).bool()
background_patches = ~dend_patches

In [ ]:
fake, fake_mu, fake_sigma = test(model, vox_patches, style_combs[:,0], style_combs[:,1])

In [ ]:
gauss_sz = np.ceil((np.array(opt.sigmas)*3*2+1)).astype(int)
gauss_sz += 1-gauss_sz%2
if opt.in_dim=='2D':
    gauss_sz[0] = 1

In [ ]:
vox_patches[0,:,0].shape

In [ ]:
X = sample_patches(vox_patches[0,:,0], gauss_sz, (1,1,1), padding=False)
X = X.view(-1, np.prod(gauss_sz)).T

Y = fake.view(len(style_combs),-1)

Ypred = no_padding((fake*dend_patches)[:,:,:,0], gauss_sz)
Ypred = Ypred.reshape(len(style_combs),-1).T

Y = no_padding((gt_patches*dend_patches)[:,:,:,0], gauss_sz)
Y = Y.reshape(len(style_combs),-1).T

inv_covar = torch.inverse(torch.matmul(X,X.T))
cross_corr_pred = torch.matmul(X,Ypred)
cross_corr = torch.matmul(X,Y)

w_pred = torch.matmul(inv_covar,cross_corr_pred)
w_pred = (w_pred/w_pred.sum(0)).T

w = torch.matmul(inv_covar,cross_corr)
w = (w/w.sum(0)).T

In [ ]:
MSE_gauss = ((w-w_pred)**2).mean(1)
hist_dendrites = histogram_intersection(fake[:,dend_patches[0]],gt_patches[:,dend_patches[0]])
hist_background = histogram_intersection(fake[:,background_patches[0]],gt_patches[:,background_patches[0]])

In [ ]:
powers = np.linspace(min(opt.powers), max(opt.powers), num=opt.n_interp)
frames = opt.n_interp*[min(opt.frames)]

fake, fake_mu, fake_sigma = test(model, vox_patches, powers, frames)
avgs = fake.mean([-1,-2,-3,-4,-5])
_, _, r_value, _, _ = stats.linregress(powers, avgs)

In [ ]:
powers = opt.n_frames*[max(opt.powers)]
frames = opt.n_frames*[min(opt.frames)]

fake, fake_mu, fake_sigma = test(model, vox_patches, powers, frames)
var = fake[:,dend_patches[0]].var(1)

In [ ]:
fake[:,dend_patches[0]].var(1)

In [ ]:
def visualize_grid(model, vox, file, opt, gt=False):
    to_grid = partial(vutils.save_image, nrow=len(opt.frames), padding=2, normalize=False, pad_value=1.0)
    
    style_combs = np.asarray(list(itertools.product(opt.powers, opt.frames)))
    input = np.concatenate(len(style_combs)*[vox[None]])
    z_ind = np.argmax(vox.sum(-1).sum(-1))

    data = {'mesh_semantics': input[None],
            'power': style_combs[:,0],
            'frames': style_combs[:,1]}
    data = tensorize_dict(data)

    if gt:
        fake = generate(vox, opt.sigmas, opt.background, opt.p_blobs, style_combs[:,0], style_combs[:,1])
        to_grid(fake[:,:,z_ind], file+'_fake.png')
        return

    fake, fake_mu, fake_sigma = model(data,'inference')

    to_grid(fake[:,:,z_ind], file+'_fake.png')
    to_grid(fake_mu[:,:,z_ind], file+'_mu.png')
    to_grid(fake_sigma[:,:,z_ind], file+'_sigma.png')

In [ ]:
[len(powers),**vox_patches.shape[1:]]

In [ ]:
x = [1,2,3,4,5,6]

In [ ]:
np.asarray(x)//2

In [ ]:
opt.name